## Download model weights and JackHMMer binary and place in Unity Catalog Volume

In [0]:
CATALOG = 'protein_folding'
SCHEMA = 'boltz'
WEIGHTS_VOLUME = 'weights'
BINARIES_VOLUME = 'binaries' 

import os
os.environ['CATALOG'] = CATALOG
os.environ['SCHEMA'] = SCHEMA
os.environ['WEIGHTS_VOLUME'] = WEIGHTS_VOLUME
os.environ['BINARIES_VOLUME'] = BINARIES_VOLUME

spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.{WEIGHTS_VOLUME}")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.{BINARIES_VOLUME}")

### Download Weights place in Volumes

In [0]:
import urllib
from pathlib import Path
CCD_URL = "https://huggingface.co/boltz-community/boltz-1/resolve/main/ccd.pkl"
MODEL_URL = (
    "https://huggingface.co/boltz-community/boltz-1/resolve/main/boltz1_conf.ckpt"
)

def download(cache: Path) -> None:
    """Download all the required data.

    Parameters
    ----------
    cache : Path
        The cache directory.

    """
    # Download CCD
    ccd = cache / "ccd.pkl"
    if not ccd.exists():
        # click.echo(
        #     f"Downloading the CCD dictionary to {ccd}. You may "
        #     "change the cache directory with the --cache flag."
        # )
        urllib.request.urlretrieve(CCD_URL, str(ccd))  # noqa: S310

    # Download model
    model = cache / "boltz1_conf.ckpt"
    if not model.exists():
        # click.echo(
        #     f"Downloading the model weights to {model}. You may "
        #     "change the cache directory with the --cache flag."
        # )
        urllib.request.urlretrieve(MODEL_URL, str(model)) 

In [0]:
CACHE_DIR = f'/Volumes/{CATALOG}/{SCHEMA}/{WEIGHTS_VOLUME}'
download(Path(CACHE_DIR))

### Download jackhmmer binary and place in Volumes

In [0]:
%sh

if [ ! -d "/miniconda3" ]; then
  mkdir -p /miniconda3

  wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O /miniconda3/miniconda.sh
  bash /miniconda3/miniconda.sh -b -u -p /miniconda3
  rm -rf /miniconda3/miniconda.sh
fi

source /miniconda3/bin/activate
conda config --remove channels defaults

conda create -n jackhmmer_env python=3.8 --yes
conda activate jackhmmer_env
conda install -y bioconda::hmmer

cp /miniconda3/envs/jackhmmer_env/bin/jackhmmer /Volumes/${CATALOG}/${SCHEMA}/${BINARIES_VOLUME}